In [128]:
import sys
sys.executable
!{sys.executable} -m pip install nltk 

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 1.4 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 2.2 MB/s  eta 0:00:01
     |████████████████████████████████| 298 kB 51.8 MB/s eta 0:00:01
     |████████████████████████████████| 675 kB 35.8 MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434674 sha256=8ec8c34e9f4e2eda050aa7f737d90d02e540d6743415b37eaceb72abdd95798a
  Stored in directory: /root/.cache/pip/wheels/a4/11/6d/2a069739a9961b3127c91a6d792fe8af6f53d93cfc1f234f60
Successfully built nltk


In [1]:
import swifter
import pandas as pd
import numpy as np
import base64
from tqdm.notebook import tqdm 
from scipy.spatial.distance import cdist

In [2]:
!ls ../zq/KDD2020Multimodalities-master/data

ls: 无法访问'../zq/KDD2020Multimodalities-master/data': 没有那个文件或目录


In [3]:
!ls ../data/RAW_DATA

multimodal_labels.txt  train_add_negative_sample.pkl  valid_answer.json
testA.pkl	       train.pkl		      valid.pkl
testA.tsv	       train.tsv		      valid.tsv


In [4]:
!ls ../data/PRO_DATA

ls: 无法访问'../data/PRO_DATA': 没有那个文件或目录


In [5]:
!free -g

              总计         已用        空闲      共享    缓冲/缓存    可用
内存：         503         224         195          65          83         211
交换：           1           0           1


In [6]:
data=pd.read_csv('../data/RAW_DATA/multimodal_labels.txt',sep='\t')
multimodal_labels={}
for i in range(len(data)):
    multimodal_labels[data.loc[i,'category_id']]=data.loc[i,'category_name']
multimodal_labels 

{0: 'top clothes (coat, jacket, shirt, etc.)',
 1: 'skirt & dress',
 2: 'bottom clothes (trousers, pants, etc.)',
 3: 'luggage, leather goods',
 4: 'shoes',
 5: 'accessories (jewelry, clothing accessories, belts, hats, scarves, etc.)',
 6: 'snacks, nuts, liquor and tea',
 7: 'makeup, perfume, beauty tools and essential oils',
 8: 'bottle drink',
 9: 'furniture',
 10: 'stationery',
 11: 'household electrical appliances',
 12: 'home decoration',
 13: 'household fabric',
 14: 'kitchenware',
 15: 'home / personal cleaning tools',
 16: 'storage supplies',
 17: 'motorcycle, motorcycle accessories, vehicles, bicycle and riding equipment',
 18: 'outdoor product',
 19: 'lighting',
 20: 'toys',
 21: 'underwear',
 22: 'digital supplies',
 23: 'bed linens',
 24: 'baby products',
 25: 'personal care',
 26: 'sporting goods',
 27: 'clothes (accessories, baby clothing, etc.)',
 28: 'others',
 29: 'human face',
 30: 'arm',
 31: 'hair',
 32: 'hand'}

In [14]:
train=pd.read_csv('../data/RAW_DATA/train.tsv',sep='\t',quoting=3,usecols=['query'])
train=pd.DataFrame(list(set(train['query'].tolist())),columns=['query'])
train.head(5)

,query
0,tea all solid wood sofa
1,everyday casual middle-aged low-top shoes
2,flagship store autumn and winter jacket
3,herbal distilled oil soap
4,classic wool vest


In [15]:
# 里面有很多的 - 符号
# 看看各种奇怪的标点

interpunctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%']   #定义标点符号列表
interpunctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%'] 
for x in tqdm(set(train['query'].tolist())):
    cutwords2 = [word for word in x if word in interpunctuations]   #去除标点符号
    if cutwords2!=[]:
        print(x)    

ms. western style shirt
stainless steel solingen, tools suit
ms. protection sleeve cuff
loose version of gold velvet flare & wide-leg
ms. no. shoulder backpack
minimalist casual ms. japan and south korea watch
plastic no. box
sleep aids ms. wine
ms. messenger storage bag
men ms. men's bracelet
to increase no. out storage bag
comfortable ms. low shoes
japan and south korea ms. shoulder backpack
ms. knitted pants
thick to increase no. stool
because ms. perfume
trendy ms. back bag
2019 ms. boots
large no. huge pooper scooper
solar no. calculator
ms. rose lotion
baby calcium calcium and iron, zinc
refreshing 3 no. foundation
ms. plus velvet coat
3 no. short-sleeved t-shirt
baby calcium and iron, zinc
no. ms. shoulder backpack
hikuma ms. boots
old altar lemon, canned
aloe hand, foot and cleansing
3 no. soothing facial cream
heel ms. socks
jamie men ms. foot agent
retro large no. storage box
in low waisted ms. pants
ms. flash diamond ring
45 no. cold-drawn steel
moving large no. storage bag


19 no. big wave volume straight hair
ms. lace umbrella
industrial large no. steel ball
sports ms. back bag
men ms. student t-shirt
to correct cute correction fluid, correction fluid
ms. loose women's
breathable ms. shoe covers
ms. wash hanger
45 no. large diameter pipe
in the elderly ms. pants
handmade ms. ornaments
no. cotton pajamas suit
association, where cream care suit
thick ms. plate
ms. blue striped shirt
ms. direct mail european and american watch
waterproof ms. raincoat
girl ms. t-shirt
mall same paragraph ms. underwear
ms. high waisted skirts
ms. imported pot
casual ms. hat
february ms. suit
crown ms. perfume
ms. fluffy hair style
ms. buckle shoes
coral velvet ms. at home robe
11 no. natural ring
straight loose flare & wide-leg
first layer of leather ms. boots
tassel ms. skirts
shu uemura hold & facial cream
long-sleeved ms. dress
2 no. model toys
four treasures no. paperweight
sleeveless 14 no. dress
construction bank ms. hat
cooked frozen emperor king crab, king crab
ms. ho

instant, green beans, fruits and vegetables dry
ms. delicate perfume
engels, books
jade 1883 no. jewelry stone
classic ms. boots
ms. retro wallet
7 no. casual pants
modified 5 no. accessories
2519 no. natural ornaments
ms. creative socks
ms. high school wallet
no. children notepad
counter money ms. back bag
triangle 7 no. banner
poplin ms. shirt
ms. trendy bracelet
no. dusting brush
ms. british woolen jacket
ms. v-neck t-shirt
gold-plated ms. necklace
mesh ms. sweater
cotton to increase no. pajamas suit
three pairs of ms. insole
acrylic sound, and magnetic alarm
with no. ethnic stamp
dress ms. clothing suit
coral velvet ms. pajamas suit
ms. korean version of goggles
ms. girls underwear
ms. cute low-top shoes
single-breasted ms. jacket
ms. double-sided hat
ms. glossy bracelet
wild head, face, head, hair accessories
domestic ms. casual pants
ms. korean series wallet
ms. embroidery scarf shawl
ms. dome hat
bigfoot significantly higher flare & wide-leg
power men ms. shampoo
short paragraph

mr. four seasons books
deep neckline ms. shirt
large no. washable lazy sofa
mechanical ms. japan and south korea watch
ms. limited perfume
long-sleeved ms. down jacket
wedding large no. box
cucumber russian, canned
printing ms. warm suit
41 no. pasta
oufu ms. cleansing
ms. cloth shoulder backpack
ms. combination of wine
children's wolf mr. books
stitching ms. underwear
ultra-light super-hard 80 no. fishing rod fishing
ms. sports scarf
sports men ms. insole
waterproof ms. shower cap
ms. wild bracelet
original 24 no. lighter
ms. yan blue bracelet
: dragon palm activity model toys
ms. party hair accessories
small fresh 20.18 million suitcase
5 no. luminous japan and south korea watch
sports & leisure books
calcium and iron, zinc fitness reconstituted product
instant specialty, canned
st. dior letters jacket
waterproof 5 no. banner
ms. quality cotton underwear
ms. cat eye glasses frame
ms. long-sleeved golf ball gown
imitation wool men ms. insole
boucheron ms. perfume
one pair of male ms. 

home, leather
knitted ms. hat
ms. refund belt
2019 day ms. back bag
merlin, bamboo and chrysanthemum sterling silver tea caddy
ms. myopia frame glasses frame
ms. short-sleeved brooch
ms. thick gloves
ms. naomi ring
ms. lazy slippers
trendy ms. low shoes
sexy ms. boots
02 no. no bleach coffee filter paper
ms. hook casual pants
no box ms. perfume
pink crystal ms. perfume
men ms. and nephrite necklaces
family large no. playing in the water toys
male ms. rosary
citrus ms. perfume
hi mary baking, canned
engraving $ stamp
fashion wild ms. back bag
original ms. jewelry clasp
men ms. guan gong necklaces
in fact, work books
ms. portable back bag
san di soft, sensitive facial cream
male ms. a wooden bracelet
jade 3 no. necklaces
instant begonia, canned
korean version of fashion flare & wide-leg
trendy ms. watch
ms. half frame glasses frame
temperament ms. earrings
to increase no. tendon at the end low shoes
17 no. alloy model toys
24 no. sports basketball socks
storage no. tea caddy
wear and rub

...

embossed ms. belt
waterproof 5 no. flashlight
ms. ethnic style gloves
sponge ms. insole
travel male ms. shoulder backpack
ms. lightweight hiking shoes
ms. thick with low-top shoes
gorgeous ms. shirt
12 no. home blender
free punch structure glass, plastic
001 no. curtain hook
grape, canned
short paragraph ms. suit
silicone neutral glass, plastic
large no. pedal rodent control
ribbon ms. belt
no. european ornaments
handmade, animal husbandry, heart seal
casual thick flare & wide-leg
ms. sexy swimsuit
baby bear calcium and iron, zinc
sports, barcelona, hat
silicon, phosphorus crystal pre-water purifier
diamond ms. belt
ms. side of the hair accessories
one two three no. wolf plush pen
autumn 20.19 million canvas shoes
ms. street beat canvas shoes
8 no. special underwear suit
ms. chiffon dress
ethnic style ms. shoulder backpack
plastics, dining tablecloth
male ms. fashion glasses frame
no. Chanel's style hair accessories
non-gm jiangsu, zhejiang sauce seasoning
ms. fashion slippers
ms. wris

warm male ms. high-top shoes
ms. body wash shaver
body ms. sunscreen cream
mrs. japan and south korea earrings
long sections ms. wallet
athletics ms. sports socks
minimalist jiangsu, zhejiang curtain
37 no. nail nail makeup
metal 4 no. zip
10 no. ziplock
la liga season, football clothes
99 no. natural gourd products
ji yue male ms. cup
3 no. net through the lotion
fabric large no. chopping
ms. korean version of hair accessories
spider-man large no. piggy bank
123 no. natural jewelry stone
7 no. color line accessories
ms. does not fade necklaces
dr. wei cream lotion
anti-oil splash ms. kitchen suit
pregnant women korean version of fashion flare & wide-leg
small sling ms. vest
accessories ms. necklace
5 no. retro envelope
ms. letters back bag
2019, canned
ms. cotton handkerchief
small floral ms. rain boots
ms. sexy low-top shoes
multifunction ms. wallet
exam ms. watch
youth dr. chen facial cream
sweet ms. low shoes
ms. riding helmet
western style ms. clothing
winter men ms. scarf shawl
d

fashion casual men ms. european and american watch
ms. cute scarf shawl
modal thin section flare & wide-leg
casual metro, canned
cream zi conditioning, ginger
polka dot ms. pajamas suit
children's 5 no. battery
big day, if the home home ornaments
certain discoloration) books
todd ms. back bag
minimalist e. hair accessories
7 no. folding lamps
ms. leather slippers
classic ms. low shoes
natural male ms. hand cream
first layer of leather ms. shoulder backpack
ms. knitted dress
lycra 47.3 million single power micro single
ms. plastic hair accessories
ms. nursing underwear
5 no. single-sided development accessories
ms. japanese canvas shoes
ms. citrus preparation of wine
thick ms. hanger
ms. sexy garter
46 no. mechanical industrial lubricants
short paragraph ms. gloves
free punch hook glass, plastic
ms. thin section underwear
giesecke & devrient postcard
mummy fashion flare & wide-leg
pleated ms. skirts
ms. high collar jacket
ms. home kitchen knife
large ms. shoulder backpack
instrument ms.

jiangsu, zhejiang eighth grade books
waist ms. big yards women's
ms. butterfly glasses
ms. light perfume
speed hair ms. casual pants
ms. direct mail outdoor clothing
food-grade glass, plastic
japanese version of the 133 no. model toys
ms. cute cup
ms. breathable back bag
geometric ms. necklaces
wind ms. scarf shawl
housewife ms. jacket
ms. bottoming socks
stainless steel 811 no. cup
letters ms. fastening
furnishings large no. piggy bank
jiangsu, zhejiang books
ms. optional journals
ms. miss necklace
mr. to blue flower
s no. no. no. small yards casual pants
wooden merlin, bamboo and chrysanthemum paperweight
ms. leather wallet
thick jiangsu, zhejiang shopping cart
mr. men's shirt
denim ms. skirts
ms. everbright suit
6 no. accessories
digital 14 no. dice
ms. casual european and american watch
13 no. crooked mouth hooks
8 no. resin zip
hong love ms. perfume
ms. minimalist fashion earrings
5 no. leather envelope
to increase no. autumn and winter hair accessories
st. mary stamp
shoulder ms.

32 no. game machines
fashion ms. latin dance shoes
down large no. cloth paste
at hem ms. dress
shallow mouth ms. socks
with long hair ms. wig
mother paragraph ms. underwear
ms. cute storage bag
trendy pin tong, then headset
ms. grain cereal
atmosphere ms. clothing suit
ms. in the elderly women's
ms. imported wine
direct mail ms. facial cream
merlin, bamboo and chrysanthemum decorative painting
long-sleeved ms. clothing suit
ms. small fresh smart bracelet
full leather ms. low shoes
ms. striped sports pants
yin and yang, yin and yang bags
ms. daughter pants
long section of ms. coat
3 no. ball machine
16 no. more meat
go hand in hand mr. storage box
temperament ms. back bag
men ms. fever warm suit
ms. bear socks
4 no. junior high school football
ms. thin section pajamas suit
drink ms. preparation of wine
trendy brand ms. back bag
ms. loving home cup
st. state stool
ms. pull face beauty tools
fashion shoes, home
ms. collar vest
6 no. and comfortable underwear
slim ms. warm suit
live ms. wa

In [16]:
# 对词进行分割
train['split_query']=train['query'].apply(lambda x:x.split(' '))
train.head(5)

,query,split_query
0,tea all solid wood sofa,"[tea, all, solid, wood, sofa]"
1,everyday casual middle-aged low-top shoes,"[everyday, casual, middle-aged, low-top, shoes]"
2,flagship store autumn and winter jacket,"[flagship, store, autumn, and, winter, jacket]"
3,herbal distilled oil soap,"[herbal, distilled, oil, soap]"
4,classic wool vest,"[classic, wool, vest]"


In [17]:
# 最后一个词
# 最后一个词里面有些会出现 ‘-’ 需要分割

# 1、 十字绣 改一下
# 2、 tshirt 改一下
# 3、o-o- pump- 改一下

def fun(x):
    # 修正一下
    if x[-1]=='cross-stitch':
        x.pop(-1)
        x.append('cross') 
        x.append('embroidery')
    elif x[-1]=='t-shirt':
        x[-1]='shirt'
    elif x[-1]=='o-o-':
        x.pop(-1)
    elif x[-1]=='pump-':
        x[-1]='pump'
    
    
    if '-' in x[-1]:
        ans=x[-1].split('-')[-1]
        if len(ans)<3:  # 奇怪的东西  比如 o-o- pump-
            print(x,ans)
    else:
        ans=x[-1]
    return ans 

train['last_query_word']=train['split_query'].apply(fun)
train
# cd 词向量里面有 问题不大

['original', 'self-cd'] cd


,query,split_query,last_query_word
0,tea all solid wood sofa,"[tea, all, solid, wood, sofa]",sofa
1,everyday casual middle-aged low-top shoes,"[everyday, casual, middle-aged, low-top, shoes]",shoes
2,flagship store autumn and winter jacket,"[flagship, store, autumn, and, winter, jacket]",jacket
3,herbal distilled oil soap,"[herbal, distilled, oil, soap]",soap
4,classic wool vest,"[classic, wool, vest]",vest
...,...,...,...
1351070,long shoe store changing his shoes,"[long, shoe, store, changing, his, shoes]",shoes
1351071,minimalist with a ornaments,"[minimalist, with, a, ornaments]",ornaments
1351072,printing crocodile scarf shawl,"[printing, crocodile, scarf, shawl]",shawl
1351073,bamboo fiber national roll,"[bamboo, fiber, national, roll]",roll


In [18]:
unique_last_query_word=set(train['last_query_word'].tolist())
print(len(train['last_query_word'].tolist()))
print(len(unique_last_query_word))

1351075
2068


In [24]:
# 加载词向量
from gensim.models.keyedvectors import KeyedVectors
gensim_model = KeyedVectors.load_word2vec_format(
    'GoogleNews-vectors-negative300.bin', binary=True)

In [11]:
# print('hello =', gensim_model['cd'])  #查看hello对应的词向量

# 计算了最后一个词的相似度

In [25]:
# 在dataframe上操作unique query last word 最后merge
unique_last_query_word=pd.DataFrame(unique_last_query_word,columns=['last_query_word'])

# 寻找出 word2vec
unique_last_query_word['last_word_embedding']=np.empty((len(unique_last_query_word), 0)).tolist()
unique_last_query_word['not_find_word']=0

for i in tqdm(range(len(unique_last_query_word))):
    try:
        unique_last_query_word.at[i,'last_word_embedding']=list(gensim_model[unique_last_query_word.loc[i,'last_query_word']])
    except:
        unique_last_query_word.at[i,'last_word_embedding']=list(np.zeros((300, ), dtype='float32'))
        unique_last_query_word.at[i,'not_find_word']=1
        print(unique_last_query_word.loc[i,'last_query_word'])
print(len(unique_last_query_word[unique_last_query_word['not_find_word']==1]))

# 这里去掉了找不到的单词,最终在merge的时候都被去掉了【大概去掉了5k条数据】
unique_last_query_word=unique_last_query_word[unique_last_query_word['not_find_word']==0]   
unique_last_query_word

shoumei
hornsey
ershao
ski/snowboard
a
ruanxiao
nguyen
xdsl
(postcard
dangshen
qipan
naipi
xidan
15psn
baoxie
qiuyi
dianhong
tianma
ms.
liuqin
dingtalk
wobi
torreya
of
motherwort
busha
zaojiao
chongcha
aigui
hulusi
jinjunmei
naisu
weila
neiku
sanhuajiu
panwan
bianzhi
susheng
and
49psn
wowo
pinai
ciwan
zao
cento
vatan
lili
hanjin
hericium
tousheng
uplay
mmcosplay
hecu
poria
birkenstocks
longjing
cpucpu
goutouzao
xun
banhu
masheng
xueyan
)
schisandra
dimo

65


,last_query_word,last_word_embedding,not_find_word
0,utensils,"[-0.23730469, 0.0024414062, -0.0126953125, 0.1...",0
1,walkman,"[0.09375, -0.12109375, -0.029541016, 0.2089843...",0
2,faceplate,"[-0.026977539, 0.07714844, -0.26757812, -0.294...",0
3,rose,"[-0.09472656, 0.060791016, -0.16308594, -0.018...",0
4,stone,"[0.022216797, 0.21972656, -0.003479004, 0.1435...",0
...,...,...,...
2063,sofa,"[0.23144531, -0.16210938, 0.12792969, 0.298828...",0
2064,wai,"[-0.075683594, 0.017089844, 0.091796875, 0.271...",0
2065,pier,"[-0.091796875, 0.061767578, -0.18164062, -0.23...",0
2066,computer,"[0.107421875, -0.20117188, 0.123046875, 0.2119...",0


In [26]:
# 计算相互之间距离
distance=cdist(np.asarray(unique_last_query_word['last_word_embedding'].tolist()),np.asarray(unique_last_query_word['last_word_embedding'].tolist()))
print('distance calculating finish')
#去掉了自己的最近距离,剩下的按照index排序，   只取前50个相似
unique_last_query_word['last_word_similarity_index']=[list(x) for x in np.argsort(distance,axis=1)[:,1:50]]   
unique_last_query_word.reset_index(drop=True,inplace=True)
unique_last_query_word.head(5)

distance calculating finish


,last_query_word,last_word_embedding,not_find_word,last_word_similarity_index
0,utensils,"[-0.23730469, 0.0024414062, -0.0126953125, 0.1...",0,"[824, 1940, 1899, 1277, 1845, 1588, 757, 626, ..."
1,walkman,"[0.09375, -0.12109375, -0.029541016, 0.2089843...",0,"[67, 597, 1940, 1355, 203, 480, 246, 1547, 877..."
2,faceplate,"[-0.026977539, 0.07714844, -0.26757812, -0.294...",0,"[1382, 86, 1940, 335, 1845, 203, 1799, 201, 78..."
3,rose,"[-0.09472656, 0.060791016, -0.16308594, -0.018...",0,"[1988, 1011, 1579, 1457, 412, 1347, 1798, 1915..."
4,stone,"[0.022216797, 0.21972656, -0.003479004, 0.1435...",0,"[1701, 1494, 1011, 1845, 626, 1968, 1457, 1579..."


In [27]:
# index 转化为last word
unique_last_query_word['last_word_similarity']=np.empty((len(unique_last_query_word), 0)).tolist()
for i in tqdm(range(len(unique_last_query_word))):
    unique_last_query_word.at[i,'last_word_similarity']=[unique_last_query_word.loc[x,'last_query_word'] for x in unique_last_query_word.loc[i,'last_word_similarity_index']]
unique_last_query_word.head(5)

,last_query_word,last_word_embedding,not_find_word,last_word_similarity_index,last_word_similarity
0,utensils,"[-0.23730469, 0.0024414062, -0.0126953125, 0.1...",0,"[824, 1940, 1899, 1277, 1845, 1588, 757, 626, ...","[trays, ziplock, tableware, kitchen, cruet, fo..."
1,walkman,"[0.09375, -0.12109375, -0.029541016, 0.2089843...",0,"[67, 597, 1940, 1355, 203, 480, 246, 1547, 877...","[portable, psn, ziplock, suona, fet, stereo, m..."
2,faceplate,"[-0.026977539, 0.07714844, -0.26757812, -0.294...",0,"[1382, 86, 1940, 335, 1845, 203, 1799, 201, 78...","[keycaps, thermowell, ziplock, console, cruet,..."
3,rose,"[-0.09472656, 0.060791016, -0.16308594, -0.018...",0,"[1988, 1011, 1579, 1457, 412, 1347, 1798, 1915...","[drop, biloba, in, dendrobium, year, half, saw..."
4,stone,"[0.022216797, 0.21972656, -0.003479004, 0.1435...",0,"[1701, 1494, 1011, 1845, 626, 1968, 1457, 1579...","[marble, brick, biloba, cruet, yuba, xie, dend..."


In [28]:
train=pd.merge(train,unique_last_query_word[['last_query_word','last_word_similarity']],on='last_query_word')
train.head(5)

,query,split_query,last_query_word,last_word_similarity
0,tea all solid wood sofa,"[tea, all, solid, wood, sofa]",sofa,"[bed, pillow, mattress, dresser, duvet, ottoma..."
1,eyelashes foot sofa,"[eyelashes, foot, sofa]",sofa,"[bed, pillow, mattress, dresser, duvet, ottoma..."
2,minimalist corner sofa,"[minimalist, corner, sofa]",sofa,"[bed, pillow, mattress, dresser, duvet, ottoma..."
3,living room hedgehog sandalwood sofa,"[living, room, hedgehog, sandalwood, sofa]",sofa,"[bed, pillow, mattress, dresser, duvet, ottoma..."
4,cartoon cotton children's sofa,"[cartoon, cotton, children's, sofa]",sofa,"[bed, pillow, mattress, dresser, duvet, ottoma..."


In [29]:
train.to_pickle("../data/PRO_DATA/transfer_query.pkl")

# query 相似度

In [30]:
train=pd.read_pickle("../data/PRO_DATA/transfer_query.pkl")
print(train.shape)
train.head(5)

(1348080, 4)


,query,split_query,last_query_word,last_word_similarity
0,tea all solid wood sofa,"[tea, all, solid, wood, sofa]",sofa,"[bed, pillow, mattress, dresser, duvet, ottoma..."
1,eyelashes foot sofa,"[eyelashes, foot, sofa]",sofa,"[bed, pillow, mattress, dresser, duvet, ottoma..."
2,minimalist corner sofa,"[minimalist, corner, sofa]",sofa,"[bed, pillow, mattress, dresser, duvet, ottoma..."
3,living room hedgehog sandalwood sofa,"[living, room, hedgehog, sandalwood, sofa]",sofa,"[bed, pillow, mattress, dresser, duvet, ottoma..."
4,cartoon cotton children's sofa,"[cartoon, cotton, children's, sofa]",sofa,"[bed, pillow, mattress, dresser, duvet, ottoma..."


## 先获取query的句向量

In [31]:
count=0
def avg_feature_vector(sentence, model=gensim_model, num_features=300):  #输入 句子  返回 向量（所有词的平均）
    global count 
    count=count+1
    if count%10000==0:
        print('*'*5,sentence,count,count/1348080)
    words = sentence.split()
    #feature vector is initialized as an empty array
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in model.index2word:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return list(feature_vec)


In [32]:
# 获取每个query的word2vec
print('query word2vec starting')
train['average_vector']=train['query'].apply(lambda x:avg_feature_vector(x,gensim_model,num_features=300))  
print('query word2vec finish')

query word2vec starting
***** early spring limited edition low-top shoes 10000 0.007417957391252745
***** wild trendy brand high-top shoes 20000 0.01483591478250549
***** baby handmade young shoes 30000 0.022253872173758234
***** leopard spring and summer low-top shoes 40000 0.02967182956501098
***** slope with leather low-top shoes 50000 0.03708978695626372
***** cashmere plaid woolen jacket 60000 0.04450774434751647
***** knee tall down jacket 70000 0.05192570173876921
***** street beat new fashion jacket 80000 0.05934365913002196
***** gold velvet network red jacket 90000 0.0667616165212747
***** outdoor photography vest 100000 0.07417957391252744
***** large craft teapot 110000 0.0815975313037802
***** trendy official t-shirt 120000 0.08901548869503294
***** waves embroidery t-shirt 130000 0.09643344608628568
***** harajuku style spring shirt 140000 0.10385140347753842
***** net red and long sections shirt 150000 0.11126936086879118
***** neckline cotton t-shirt 160000 0.1186873182

In [33]:
train.to_pickle("../data/PRO_DATA/transfer_query.pkl")

相似度

In [ ]:
similarity=pd.read_pickle("../data/PRO_DATA/transfer_query.pkl")
similarity.head(5)

## 数据太多，直接计算会爆内存，根据last word在组内做计算

有些的last word也很大（9w），需要对其进行拆分

可能是因为数据太多了（部分比如说book有9w条）      
这边last word 少的就增加到1k，多于1w的就分割到1w

In [60]:
print(len(set(similarity['last_query_word'])))
similarity['last_query_word'].value_counts()

2003


books          92242
shoes          54532
shirt          52618
suit           48774
pants          41242
               ...  
eggshell           1
surrounding        1
vehicle            1
solar              1
mo                 1
Name: last_query_word, Length: 2003, dtype: int64

label=0 无需处理          
label=1 的需要后续扩充         
label=2 的减少                 

In [20]:
similarity['new_last_word']=similarity['last_query_word']
similarity.head(5)

,query,split_query,last_query_word,last_word_similarity,average_vector,new_last_word,all_index_similarity_index
0,tea all solid wood sofa,"['tea', 'all', 'solid', 'wood', 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
1,eyelashes foot sofa,"['eyelashes', 'foot', 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
2,minimalist corner sofa,"['minimalist', 'corner', 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
3,living room hedgehog sandalwood sofa,"['living', 'room', 'hedgehog', 'sandalwood', '...",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
4,cartoon cotton children's sofa,"['cartoon', 'cotton', ""children's"", 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."


In [23]:
similarity['new_last_word'].value_counts()  # 处理之前的效果

books      92242
shoes      54532
shirt      52618
suit       48774
pants      41242
           ...  
sunfish        1
cute           1
hundred        1
scores         1
round          1
Name: new_last_word, Length: 2003, dtype: int64

In [24]:
def func(x):
#     print(x['last_query_word'])
    MAXLENGTH=3000    # 如果数据》3000条 拆分
    SPLITLENGTH=2500  # 超过的按照2500分割
    if len(x)>MAXLENGTH:  
        now_last_query_word=x['new_last_word'].tolist()[0]  # 获取当前最后一个词      
        for i in range(0,len(x),SPLITLENGTH):
            length=len(x.iloc[i:i+SPLITLENGTH,:])
            x.iloc[i:i+SPLITLENGTH,5]=[now_last_query_word+'_'+str(i) for x in range(length)]  #第五个是new last word
        if len(x.iloc[i:i+SPLITLENGTH,5])<100:
            x.iloc[i:i+SPLITLENGTH,5]=[now_last_query_word+'_'+str(i-SPLITLENGTH) for x in range(length)]  #第五个是new last word
    return x
similarity=similarity.groupby(['new_last_word']).apply(func)

similarity['new_last_word'].value_counts()  # 处理之后的效果

machine      2946
frame        2926
cd           2924
pots         2876
plants       2872
             ... 
t               1
pick            1
moonlight       1
roses           1
vehicle         1
Name: new_last_word, Length: 2317, dtype: int64

In [26]:
len(set(similarity['new_last_word']))

2317

In [27]:
from scipy.spatial.distance import pdist,squareform

similarity['all_index_similarity_index'] = np.empty((len(similarity), 0)).tolist()
count=0

In [28]:
similarity.head(5)

,query,split_query,last_query_word,last_word_similarity,average_vector,new_last_word,all_index_similarity_index
0,tea all solid wood sofa,"['tea', 'all', 'solid', 'wood', 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,[]
1,eyelashes foot sofa,"['eyelashes', 'foot', 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,[]
2,minimalist corner sofa,"['minimalist', 'corner', 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,[]
3,living room hedgehog sandalwood sofa,"['living', 'room', 'hedgehog', 'sandalwood', '...",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,[]
4,cartoon cotton children's sofa,"['cartoon', 'cotton', ""children's"", 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,[]


In [29]:
# 读文件的时候读出来 average_vector 是str格式，做一下转换
def fund(x):
    return [np.float(i) for i in similarity.loc[0,'average_vector'][1:-1].split(',')]
similarity['average_vector']=similarity['average_vector'].apply(fund)

## 组内相似度计算，并排序

In [30]:
count=0
def fun(df):
    global count 
    count=count+1
    if count%50==0:
        print('*'*5,count/2317)
    if len(df)>1:
        distance=squareform(pdist(np.asarray(df['average_vector'].tolist())))
    #     df['inner_group_similarity_value']=[list(x) for x in distance]   # 按照group内index排序的相似性
    #     df['inner_group_similarity_index']=[list(x) for x in np.argsort(distance,axis=1)[:,1:1000]]   # 排列之后的  这种算出来的index是group内的index  按照顺序

        df['all_index_similarity_index']=[list([df.index[i] for i in g]) for g in np.argsort(distance,axis=1)[:,1:1000]] # 矫正之后的index才是整个数据集上的index
        # 后面和数据集合并的时候把这个index当成值，取出来
    return df
similarity=similarity.groupby(['new_last_word']).apply(fun)
similarity

***** 0.02157962883038412
***** 0.04315925766076824
***** 0.06473888649115235
***** 0.08631851532153648
***** 0.10789814415192059
***** 0.1294777729823047
***** 0.1510574018126888
***** 0.17263703064307295
***** 0.19421665947345707
***** 0.21579628830384118
***** 0.2373759171342253
***** 0.2589555459646094
***** 0.2805351747949935
***** 0.3021148036253776
***** 0.32369443245576174
***** 0.3452740612861459
***** 0.36685369011653
***** 0.38843331894691413
***** 0.41001294777729824
***** 0.43159257660768235
***** 0.45317220543806647
***** 0.4747518342684506
***** 0.4963314630988347
***** 0.5179110919292188
***** 0.5394907207596029
***** 0.561070349589987
***** 0.5826499784203711
***** 0.6042296072507553
***** 0.6258092360811394
***** 0.6473888649115235
***** 0.6689684937419076
***** 0.6905481225722918
***** 0.7121277514026759
***** 0.73370738023306
***** 0.7552870090634441
***** 0.7768666378938283
***** 0.7984462667242124
***** 0.8200258955545965
***** 0.8416055243849806
***** 0.863185153

,query,split_query,last_query_word,last_word_similarity,average_vector,new_last_word,all_index_similarity_index
0,tea all solid wood sofa,"['tea', 'all', 'solid', 'wood', 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
1,eyelashes foot sofa,"['eyelashes', 'foot', 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
2,minimalist corner sofa,"['minimalist', 'corner', 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
3,living room hedgehog sandalwood sofa,"['living', 'room', 'hedgehog', 'sandalwood', '...",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
4,cartoon cotton children's sofa,"['cartoon', 'cotton', ""children's"", 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
...,...,...,...,...,...,...,...
1348075,full copper beads touch,"['full', 'copper', 'beads', 'touch']",touch,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",touch,[]
1348076,to sleep pillow pillow papa,"['to', 'sleep', 'pillow', 'pillow', 'papa']",papa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",papa,[]
1348077,student shot in the exam,"['student', 'shot', 'in', 'the', 'exam']",exam,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",exam,[]
1348078,with lid universe,"['with', 'lid', 'universe']",universe,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",universe,[]


In [31]:
similarity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1348080 entries, 0 to 1348079
Data columns (total 7 columns):
 #   Column                      Non-Null Count    Dtype 
---  ------                      --------------    ----- 
 0   query                       1348080 non-null  object
 1   split_query                 1348080 non-null  object
 2   last_query_word             1348080 non-null  object
 3   last_word_similarity        1348080 non-null  object
 4   average_vector              1348080 non-null  object
 5   new_last_word               1348080 non-null  object
 6   all_index_similarity_index  1348080 non-null  object
dtypes: object(7)
memory usage: 122.3+ MB


In [97]:
# csv存一次
similarity.to_csv('transfer_query_add_query_embedding_similarity.csv')

In [18]:
similarity

,query,split_query,last_query_word,last_word_similarity,average_vector,new_last_word,all_index_similarity_index
0,tea all solid wood sofa,"['tea', 'all', 'solid', 'wood', 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
1,eyelashes foot sofa,"['eyelashes', 'foot', 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
2,minimalist corner sofa,"['minimalist', 'corner', 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
3,living room hedgehog sandalwood sofa,"['living', 'room', 'hedgehog', 'sandalwood', '...",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
4,cartoon cotton children's sofa,"['cartoon', 'cotton', ""children's"", 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
...,...,...,...,...,...,...,...
1348075,full copper beads touch,"['full', 'copper', 'beads', 'touch']",touch,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",touch,None
1348076,to sleep pillow pillow papa,"['to', 'sleep', 'pillow', 'pillow', 'papa']",papa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",papa,None
1348077,student shot in the exam,"['student', 'shot', 'in', 'the', 'exam']",exam,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",exam,None
1348078,with lid universe,"['with', 'lid', 'universe']",universe,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",universe,None


# n sample
计算出了相似度后需要选取合适的距离作为 negative sample

In [3]:
similarity=pd.read_csv('transfer_query_add_query_embedding_similarity.csv',index_col=0)
similarity

,query,split_query,last_query_word,last_word_similarity,average_vector,new_last_word,all_index_similarity_index
0,tea all solid wood sofa,"['tea', 'all', 'solid', 'wood', 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
1,eyelashes foot sofa,"['eyelashes', 'foot', 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
2,minimalist corner sofa,"['minimalist', 'corner', 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
3,living room hedgehog sandalwood sofa,"['living', 'room', 'hedgehog', 'sandalwood', '...",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
4,cartoon cotton children's sofa,"['cartoon', 'cotton', ""children's"", 'sofa']",sofa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
...,...,...,...,...,...,...,...
1348075,full copper beads touch,"['full', 'copper', 'beads', 'touch']",touch,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",touch,[]
1348076,to sleep pillow pillow papa,"['to', 'sleep', 'pillow', 'pillow', 'papa']",papa,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",papa,[]
1348077,student shot in the exam,"['student', 'shot', 'in', 'the', 'exam']",exam,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",exam,[]
1348078,with lid universe,"['with', 'lid', 'universe']",universe,"['bed', ""'pillow"", ""'mattress"", ""'dresser"", ""'...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",universe,[]


In [4]:
# 读文件出错，处理一下
def func(x):
    if x == '[]':
        return []
    else:
        return [int(i) for i in x[1:-1].split(',')]
similarity['all_index_similarity_index']=similarity['all_index_similarity_index'].apply(func)

def fund(x):
    return [i[1:-1] for i in similarity.loc[0,'last_word_similarity'][1:-1].split(',')]
similarity['last_word_similarity']=similarity['last_word_similarity'].apply(fund)

similarity

,query,split_query,last_query_word,last_word_similarity,average_vector,new_last_word,all_index_similarity_index
0,tea all solid wood sofa,"['tea', 'all', 'solid', 'wood', 'sofa']",sofa,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
1,eyelashes foot sofa,"['eyelashes', 'foot', 'sofa']",sofa,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
2,minimalist corner sofa,"['minimalist', 'corner', 'sofa']",sofa,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
3,living room hedgehog sandalwood sofa,"['living', 'room', 'hedgehog', 'sandalwood', '...",sofa,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
4,cartoon cotton children's sofa,"['cartoon', 'cotton', ""children's"", 'sofa']",sofa,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141..."
...,...,...,...,...,...,...,...
1348075,full copper beads touch,"['full', 'copper', 'beads', 'touch']",touch,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",touch,[]
1348076,to sleep pillow pillow papa,"['to', 'sleep', 'pillow', 'pillow', 'papa']",papa,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",papa,[]
1348077,student shot in the exam,"['student', 'shot', 'in', 'the', 'exam']",exam,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",exam,[]
1348078,with lid universe,"['with', 'lid', 'universe']",universe,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",universe,[]


In [5]:
# 可视化看一下
aaa=similarity[similarity['query']=='theory books']['all_index_similarity_index'].values[0]
similarity[similarity.index.isin(aaa)]['query'].tolist()



['classical characters books',
 '2nd third grade books',
 'bookworm someone books',
 'supply the power books',
 'work review books',
 'mechanical david books',
 'primary school line books',
 'prodigy hardcover books',
 'fun skills books',
 "children's intelligence books",
 'european and american classic books',
 'value edition selling books',
 'failure beautiful books',
 'teacher club parent-child books',
 'beautiful casual books',
 'french high school students books',
 'ritual monument books',
 'training massage books',
 'instance mechanical books',
 "children's zhou guoping zhou guoping books",
 'learning california books',
 'award-winning high quality books',
 'collection contact books',
 'bid books',
 'flower classic books',
 'compensation reactive power books',
 'original lace books',
 'leadership books',
 'pinyin little penguin books',
 'river pure books',
 'dong fa seven five books',
 'perseverance books',
 'children gold books',
 'high-pressure control books',
 'songs parent-ch

In [6]:
print('ok')

ok


In [10]:
similarity['n_sample_index']=np.empty((len(similarity), 0)).tolist()  # 创建一个新的列
# similarity['n_sample_index']=''
similarity

,query,split_query,last_query_word,last_word_similarity,average_vector,new_last_word,all_index_similarity_index,n_sample_index
0,tea all solid wood sofa,"['tea', 'all', 'solid', 'wood', 'sofa']",sofa,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141...",[]
1,eyelashes foot sofa,"['eyelashes', 'foot', 'sofa']",sofa,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141...",[]
2,minimalist corner sofa,"['minimalist', 'corner', 'sofa']",sofa,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141...",[]
3,living room hedgehog sandalwood sofa,"['living', 'room', 'hedgehog', 'sandalwood', '...",sofa,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141...",[]
4,cartoon cotton children's sofa,"['cartoon', 'cotton', ""children's"", 'sofa']",sofa,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141...",[]
...,...,...,...,...,...,...,...,...
1348075,full copper beads touch,"['full', 'copper', 'beads', 'touch']",touch,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",touch,[],[]
1348076,to sleep pillow pillow papa,"['to', 'sleep', 'pillow', 'pillow', 'papa']",papa,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",papa,[],[]
1348077,student shot in the exam,"['student', 'shot', 'in', 'the', 'exam']",exam,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",exam,[],[]
1348078,with lid universe,"['with', 'lid', 'universe']",universe,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",universe,[],[]


In [11]:
similarity['new_last_word'].value_counts()

machine        2946
frame          2926
cd             2924
pots           2876
plants         2872
               ... 
wipe              1
arrangement       1
exam              1
shallot           1
interlock         1
Name: new_last_word, Length: 2317, dtype: int64

In [12]:
import random
print('start')
def get_n_sample(df):
    global similarity
    # 最简单的  直接取后面80%
    MIN=int(len(df.iat[0,-2])*0.8) 
    MAX=int(len(df.iat[0,-2])*0.8)+10 

    simi_ans=similarity[similarity.index.isin(df.iat[0,-2][MIN:MAX])]['query'].tolist()

    if (simi_ans==[]) or (len(df)<5):
        simi_ans=[random.choice(similarity['query']) for i in range(MAX-MIN)]  # 没有的话就随机选
    df['n_sample_index']= [simi_ans for g in range(len(df))] 
    return df
similarity=similarity.groupby(['new_last_word']).apply(get_n_sample)
similarity

start


,query,split_query,last_query_word,last_word_similarity,average_vector,new_last_word,all_index_similarity_index,n_sample_index
0,tea all solid wood sofa,"['tea', 'all', 'solid', 'wood', 'sofa']",sofa,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141...","[private multifunction lazy sofa, wood great w..."
1,eyelashes foot sofa,"['eyelashes', 'foot', 'sofa']",sofa,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141...","[private multifunction lazy sofa, wood great w..."
2,minimalist corner sofa,"['minimalist', 'corner', 'sofa']",sofa,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141...","[private multifunction lazy sofa, wood great w..."
3,living room hedgehog sandalwood sofa,"['living', 'room', 'hedgehog', 'sandalwood', '...",sofa,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141...","[private multifunction lazy sofa, wood great w..."
4,cartoon cotton children's sofa,"['cartoon', 'cotton', ""children's"", 'sofa']",sofa,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",sofa,"[1425, 1424, 1423, 1422, 1421, 1420, 1419, 141...","[private multifunction lazy sofa, wood great w..."
...,...,...,...,...,...,...,...,...
1348075,full copper beads touch,"['full', 'copper', 'beads', 'touch']",touch,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",touch,[],"[vision cartoon wall stickers, jedi educationa..."
1348076,to sleep pillow pillow papa,"['to', 'sleep', 'pillow', 'pillow', 'papa']",papa,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",papa,[],"[autumn and winter fit casual pants, design se..."
1348077,student shot in the exam,"['student', 'shot', 'in', 'the', 'exam']",exam,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",exam,[],"[polished spherical drill, halloween cute earr..."
1348078,with lid universe,"['with', 'lid', 'universe']",universe,"[bed, ""'pillow, ""'mattress, ""'dresser, ""'duvet...","[0.0060546873, 0.07409668, -0.00234375, 0.1619...",universe,[],"[assault little women's, function student tele..."


In [12]:
similarity[similarity['query']=='treble popular reed']['n_sample_index'].values

array([list(['high-quality babe reed', 'classical bamboo fiber reed', 'babe reed', 'flagship store exemption reed', 'professional flagship store reed'])],
      dtype=object)

In [13]:
similarity=similarity[['query','n_sample_index']]
similarity

,query,n_sample_index
0,tea all solid wood sofa,"[private multifunction lazy sofa, wood great w..."
1,eyelashes foot sofa,"[private multifunction lazy sofa, wood great w..."
2,minimalist corner sofa,"[private multifunction lazy sofa, wood great w..."
3,living room hedgehog sandalwood sofa,"[private multifunction lazy sofa, wood great w..."
4,cartoon cotton children's sofa,"[private multifunction lazy sofa, wood great w..."
...,...,...
1348075,full copper beads touch,"[vision cartoon wall stickers, jedi educationa..."
1348076,to sleep pillow pillow papa,"[autumn and winter fit casual pants, design se..."
1348077,student shot in the exam,"[polished spherical drill, halloween cute earr..."
1348078,with lid universe,"[assault little women's, function student tele..."


In [17]:
import pickle
data=[]
base_dir = '../data/RAW_DATA/'
mode = 'n_query_hashmap'

for i in tqdm(range(len(similarity))):
    data.append([similarity.at[i,'query'],similarity.at[i,'n_sample_index']])
data
pickle.dump(data, open(base_dir + '{}.pkl'.format(mode), 'wb'))  

# 处理成lxmert的格式

In [15]:
train=pd.read_csv('../data/RAW_DATA/train.tsv',sep='\t',quoting=3,usecols=['product_id','image_h','image_w','num_boxes','boxes','class_labels','query','query_id'])

train

KeyboardInterrupt: 

In [ ]:
# 数据格式转换函数
def fun(x):
    NUM_BOXES=x['num_boxes']
    x['boxes']=np.frombuffer(base64.b64decode(x['boxes']), dtype=np.float32).reshape(NUM_BOXES, 4)
    x['class_labels']=np.frombuffer(base64.b64decode(x['class_labels']), dtype=np.int64).reshape(NUM_BOXES)
#         x['features']=np.frombuffer(base64.b64decode(x['features']), dtype=np.float32).reshape(NUM_BOXES, 2048)
#         x['transform_class_labels']=[multimodal_labels[x] for x in x['class_labels']]

    return x 

train=train.apply(fun,axis=1)
train

In [ ]:
train=pd.merge(train,similarity[['query','n_sample_index']],on='query')
train.head(5)

In [ ]:
!free -g

In [20]:
import pickle

counter = 0
data=[]

base_dir = '../data/RAW_DATA/'
mode = 'train_add_negative_sample'

for entry in tqdm(train.values):
    product_id, image_h, image_w, num_boxes, boxes, class_labels, query, query_id, transform_class_labels,negative_query = entry
    
    data.append({
    "product_id": product_id,
    "image_h": image_h,
    "image_w": image_w,
    "num_boxes": num_boxes,
    "boxes": boxes,
    "class_labels": class_labels,
    "query": query,
    "query_id": query_id,
    "index": counter,
    "negative_query":negative_query,
    })
    counter += 1
    
#     break
# data
pickle.dump(data, open(base_dir + '{}.pkl'.format(mode), 'wb'))   

NameError: name 'train' is not defined

# check

In [2]:


import pickle
import os
data=pickle.load(open(
            os.path.join('/home/zhangqibot/proj/kdd/data/RAW_DATA/', 'train_add_negative_sample' + '.pkl'), 'rb'
))
data

[{'product_id': 102851856,
  'image_h': 338,
  'image_w': 209,
  'num_boxes': 2,
  'boxes': array([[  0.,   3., 338., 207.],
         [239.,  22., 333., 204.]], dtype=float32),
  'class_labels': array([28, 28]),
  'query': 'treble popular reed',
  'query_id': 923271,
  'index': 0,
  'n_sample_index': ['high-quality babe reed',
   'classical bamboo fiber reed',
   'babe reed',
   'flagship store exemption reed',
   'professional flagship store reed']},
 {'product_id': 101783080,
  'image_h': 80,
  'image_w': 61,
  'num_boxes': 1,
  'boxes': array([[ 5.,  5., 75., 56.]], dtype=float32),
  'class_labels': array([28]),
  'query': 'compatible ink cartridges',
  'query_id': 234031,
  'index': 1,
  'n_sample_index': ['hp toner cartridges',
   'direct supply original toner cartridges',
   'original toner cartridges',
   'emma toner cartridges',
   'nine compatible ink cartridges',
   'compatible suitable for toner cartridges',
   'compatible print toner cartridges',
   'applicable developing t